This project explores the Black-Scholes Model (BSM) for option pricing, a cornerstone of modern quantitative finance. In addition to pricing European-style options, the project delves into the calculation and analysis of option Greeks—key sensitivity measures that quantify the risk and responsiveness of an option's price to market variables. By combining financial theory with Python-based numerical techniques, this project provides both a conceptual understanding and a hands-on implementation of the BSM and Greeks, suitable for beginners and professionals in quantitative finance alike. The code is modular, with functions designed to compute individual components of the BSM formula and Greeks, making it easy to understand and adapt for various use cases.

In [ ]:
from datetime import datetime
import yfinance as yf
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd

Importing combination of libraries that are ideal for conducting quantitative finance tasks, such as option pricing, volatility analysis, and financial data visualization.

In [ ]:
def get_stock_price(stock_symbol):
    """
    Fetches the current stock price for the given stock symbol.
    
    Args:
        stock_symbol (str): The ticker symbol of the stock (e.g., TSLA, AAPL).
    
    Returns:
        float: Current stock price or None if the symbol is invalid.
    """
    try:
        # Fetch stock data
        stock = yf.Ticker(stock_symbol)
        stock_info = stock.history(period="1d")
        
        # Check if data is available
        if stock_info.empty:
            print("Invalid stock symbol or no data available.")
            return None
        
        # Extract the latest closing price
        current_price = stock_info['Close'].iloc[-1]
        return current_price 

    except Exception as e:
        print(f"Error fetching stock price: {e}")
        return None

Fetching stock price from Yahoo Finance for user defined stock.

In [ ]:
stock_symbol = str(input("Enter the stock symbol (e.g., TSLA, AAPL): ")).upper()
stock_price = get_stock_price(stock_symbol)
if stock_price is not None:
    print(f"The current stock price of {stock_symbol} is: ${stock_price:.2f}")
else:
    print("Failed to fetch stock price.")

In [ ]:

def get_expiry_dates(stock_symbol):
    """
    Fetches the options expiry dates for the given stock symbol.
    
    Args:
        stock_symbol (str): The ticker symbol of the stock (e.g., TSLA, AAPL).
    
    Returns:
        list: A list of expiry dates or an empty list if none are available.
    """
    try:
        # Fetch stock data
        stock = yf.Ticker(stock_symbol)

         # Get option expiry dates
        expiry_dates = stock.options
        if not expiry_dates:
            print("No options data available for Tesla.")
            return None

        print(f"Available Expiry Dates: {expiry_dates}")
        return expiry_dates
    except Exception as e:
        print(f"An error occurred: {e}")
        return []


Fetching available expiry dates for the user defined stock option

In [ ]:
get_expiry_dates(stock_symbol)

In [ ]:
    
def calculate_time_to_maturity(maturity_date_str):
    """
    Calculates time to maturity in years based on the user-provided maturity date.
    
    Returns:
    - T (float): Time to maturity in years.
    """
    try:
        # Get today's date
        today = datetime.today()

        # Input maturity date from the user
        maturity_date = datetime.strptime(maturity_date_str, "%Y-%m-%d")

        # Calculate the number of days to maturity
        days_to_maturity = (maturity_date - today).days

        # Check if the maturity date is valid
        if days_to_maturity < 0:
            raise ValueError("The maturity date has already passed.")
        global T
        # Calculate time to maturity in years
        T = days_to_maturity / 365
        return T

    except ValueError as e:
        print(f"Error: {e}")
        return None

Calculating time to maturity (in years) using user defined expiry date

In [ ]:

maturity_date_str = input("Enter the maturity date (YYYY-MM-DD): ")
time_to_maturity = calculate_time_to_maturity(maturity_date_str)

if time_to_maturity is not None:
    print(f"Time to maturity (T): {time_to_maturity:.4f} years")


In [ ]:

def fetch_current_risk_free_rate():
    """
    Fetches the 3-month Treasury Bill yield (symbol: ^IRX) as the current risk-free rate.
    """
    try:
        # Fetching data for 3-month Treasury Bill
        treasury_data = yf.Ticker("^IRX")
        treasury_info = treasury_data.history(period="1d")  # Most recent data
        
        if not treasury_info.empty:
            # Get the latest closing yield
            yield_rate = treasury_info["Close"].iloc[-1]  # Annualized yield in percentage
            risk_free_rate = yield_rate / 100  # Convert percentage to decimal
            return risk_free_rate
        else:
            raise ValueError("No data found for Treasury Bill yield.")
    except Exception as e:
        print(f"Error fetching risk-free rate: {e}")
        return None


Calculating risk free rate using 3-month US treasury bill

In [ ]:

risk_free_rate = fetch_current_risk_free_rate()
if risk_free_rate is not None:
    print(f"Current Risk-Free Rate (3-Month Treasury Bill): {risk_free_rate * 100:.2f}%")
else:
    print("Unable to fetch the current risk-free rate.")

Following functions fetch call and put option implied volatility respectively

In [ ]:

def call_option_IV(ticker, strike_price,exp):
    """
    Fetches the implied volatility (IV) for a specific strike price of Tesla options.
    
    Args:
    - ticker (str): The stock ticker symbol (e.g., "TSLA" for Tesla).
    - strike_price (float): The strike price of the option.

    Returns:
    - IV (float): The implied volatility at the given strike price, or None if not found.
    """
    try:
        # Fetch Tesla stock data
        stock = yf.Ticker(ticker)

        # Get the nearest expiration date
        expiration_dates = stock.options
        if not expiration_dates:
            raise ValueError("No options data available for this ticker.")
        # user defined expiration
        #print(f"Using given expiration date: {exp}")

        # Fetch options chain for the nearest expiration
        options_chain = stock.option_chain(exp)
        calls = options_chain.calls
        puts = options_chain.puts

        # Search for the strike price in calls
        call_option = calls[calls['strike'] == strike_price]
        if not call_option.empty:
            iv = call_option.iloc[0]['impliedVolatility']
            #print(f"Implied Volatility (Calls) for strike {strike_price}: {iv:.2%}")
            return iv

        # Search for the strike price in puts if not found in calls
        print(f"No options found for strike price $1{strike_price}.")
        return None

    except Exception as e:
        print(f"Error fetching IV: {e}")
        return None

def put_option_IV(ticker, strike_price,exp):
    """
    Fetches the implied volatility (IV) for a specific strike price of Tesla options.
    
    Args:
    - ticker (str): The stock ticker symbol (e.g., "TSLA" for Tesla).
    - strike_price (float): The strike price of the option.

    Returns:
    - IV (float): The implied volatility at the given strike price, or None if not found.
    """
    try:
        # Fetch Tesla stock data
        stock = yf.Ticker(ticker)

        # Get the nearest expiration date
        expiration_dates = stock.options
        if not expiration_dates:
            raise ValueError("No options data available for this ticker.")

        nearest_expiration = expiration_dates[0]
        #print(f"Using given expiration date: {exp}")

        # Fetch options chain for the nearest expiration
        options_chain = stock.option_chain(exp)
        calls = options_chain.calls
        puts = options_chain.puts

       

        # Search for the strike price in puts if not found in calls
        put_option = puts[puts['strike'] == strike_price]
        if not put_option.empty:
            iv = put_option.iloc[0]['impliedVolatility']
            #print(f"Implied Volatility (Puts) for strike {strike_price}: {iv:.2%}")
            return iv

        print(f"No options found for strike price ${strike_price}.")
        return None

    except Exception as e:
        print(f"Error fetching IV: {e}")
        return None


In [ ]:

def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    Calculate the Black-Scholes price of an option.

    Parameters:
    - S: Current stock price
    - K: Strike price
    - T: Time to maturity (in years)
    - r: Risk-free rate (annualized)
    - sigma: Volatility (standard deviation of returns)
    - option_type: "call" or "put"

    Returns:
    - Option price (float)
    """
    try:
        # Calculate d1 and d2
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)

        if option_type == "call":
            # Call option price
            option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        elif option_type == "put":
            # Put option price
            option_price = K * np.exp(-r * T) *(1- norm.cdf(d2)) - S * (1-norm.cdf(d1))
        else:
            raise ValueError("Invalid option type. Choose 'call' or 'put'.")

        return option_price
    except Exception as e:
        print(f"Error in Black-Scholes calculation: {e}")
        return None


Calculating call and put option prices for user defined stock, expiration date and strike price using Black-Scholes-Merton Model

In [ ]:
# Calculate call and put option prices using BSM model
strike_price = float(input("Enter the strike price of the option: "))
call_price = black_scholes(get_stock_price(stock_symbol), strike_price, calculate_time_to_maturity(maturity_date_str), fetch_current_risk_free_rate(), call_option_IV(stock_symbol, strike_price,maturity_date_str), option_type="call")
put_price = black_scholes(get_stock_price(stock_symbol), strike_price, calculate_time_to_maturity(maturity_date_str), fetch_current_risk_free_rate(), put_option_IV(stock_symbol, strike_price,maturity_date_str), option_type="put")
print("\nUsing Black-Scholes-Merton Model for option pricing we get")
print(f"Call Option Price: ${call_price:.2f}")
print(f"Put Option Price: ${put_price:.2f}")


In [ ]:
# Fetch live stock and option data
def fetch_option_data(stock_symbol,maturity_date_str):
    """
    Fetches stock price and options data for a given stock symbol.
    
    Args:
        stock_symbol (str): The ticker symbol of the stock.
    
    Returns:
        dict: Dictionary containing stock price and options data.
    """
    try:
        stock = yf.Ticker(stock_symbol)
        stock_price = stock.history(period="1d")["Close"].iloc[-1]

        
        # Fetch options chain for the nearest expiry date
        options_chain = stock.option_chain(maturity_date_str)
        calls = options_chain.calls
        puts = options_chain.puts
        
        return {"stock_price": stock_price, "calls": calls, "puts": puts, "expiry_date": maturity_date_str}
    
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

# Compare BSM price to market price and calculate percentage error
def compare_prices(stock_symbol, r):
    """
    Compare BSM prices to live market option prices and compute percentage errors.
    
    Args:
        stock_symbol (str): Ticker symbol of the stock.
        r (float): Risk-free interest rate.
        sigma (float): Volatility of the stock.
    """
    data = fetch_option_data(stock_symbol,maturity_date_str)
    if not data:
        return

    stock_price = data["stock_price"]
    calls = data["calls"]
    puts = data["puts"]
    expiry_date = data["expiry_date"]
    
    
    print(f"Stock Price: ${stock_price}, Expiry Date: ${expiry_date}, Time to Expiry: {T:.4f} years")
    
    # Filter for user-defined strike price
    rowc = calls[calls["strike"] == strike_price]
    rowp = puts[puts["strike"] == strike_price]
    if rowc.empty:
        print("No data available for the specified strike price.")
        return

    # Extract market price and implied volatility
    call_market_price = rowc["lastPrice"].iloc[0]
    put_market_price = rowp["lastPrice"].iloc[0]
    # Calculate BSM price
    bsm_callprice_calculated = black_scholes(stock_price, strike_price, time_to_maturity, risk_free_rate, call_option_IV(stock_symbol, strike_price,maturity_date_str), option_type="call")
    bsm_putprice_calculated = black_scholes(stock_price, strike_price, time_to_maturity, risk_free_rate, put_option_IV(stock_symbol, strike_price,maturity_date_str), option_type="put")

    # Calculate percentage error
    call_error = abs((bsm_callprice_calculated - call_market_price) / call_market_price) * 100
    put_error = abs((bsm_putprice_calculated - put_market_price) / put_market_price) * 100
        
    print(f"Strike: ${strike_price}, Market call option price: ${call_market_price:.2f}, BSM call option Price: ${bsm_callprice_calculated:.2f}, Error: {call_error:.2f}%")
    print(f"Strike: ${strike_price}, Market put option price: ${put_market_price:.2f}, BSM put option Price: ${bsm_putprice_calculated:.2f}, Error: {put_error:.2f}%")

Comparing theoretical option price (from BSM Model) and actual option price (fetched from Yahoo finance option chain) for given strike price and finding out the percentage of error for the same

In [ ]:
compare_prices(stock_symbol, fetch_current_risk_free_rate())

In [ ]:
# Fetch option data from yfinance
def fetch_option_data(stock_symbol, expiration_date):
    stock = yf.Ticker(stock_symbol)
    option_chain = stock.option_chain(expiration_date)
    calls = option_chain.calls
    puts = option_chain.puts
    return calls, puts

# Compare calculated and actual prices
def compare_prices(stock_symbol, expiration_date, r, sigma):
    calls, puts = fetch_option_data(stock_symbol, expiration_date)
    errors = {"calls": [], "puts": []}
    print()
    for i, row in calls.iterrows():
        K = row['strike']
        sigma=call_option_IV(stock_symbol,K,maturity_date_str)
        model_price = black_scholes(get_stock_price(stock_symbol), K, calculate_time_to_maturity(maturity_date_str), r, sigma, option_type="call")
        actual_price = row['lastPrice']
        if model_price<1 and actual_price<1:
            errors["calls"].append(0)
        else:
            error = abs(model_price - actual_price) / actual_price * 100
            errors["calls"].append(error)
        print(f"Theoretical price: ${model_price:.2f}, Actual price: ${actual_price:.2f},percentage of error: {error:.2f}%")
        print(f'strike : ${K:.2f}, volatility: {sigma:.2f}%')
    for i, row in puts.iterrows():
        K = row['strike']
        sigma=put_option_IV(stock_symbol,K,maturity_date_str)
        model_price = black_scholes(get_stock_price(stock_symbol), K, calculate_time_to_maturity(maturity_date_str), r, sigma, option_type="put")
        actual_price = row['lastPrice']
        if model_price<1 and actual_price<1:
            errors["puts"].append(0)
        else:
            error = abs(model_price - actual_price) / actual_price * 100
            errors["puts"].append(error)
    return errors

# Plot errors
def plot_errors(errors):
    plt.figure(figsize=(10, 6))
    plt.plot(errors["calls"],  marker="o")
    plt.plot(errors["puts"],  marker="x")
    plt.xlabel("Option Index")
    plt.ylabel("Error Percentage (%)")
    plt.title("Error Percentage between Black-Scholes Model and Actual Option Prices")
    plt.legend()
    plt.grid()
    plt.show()

Now comparing theoretical and actual option prices for all strike prices on the option chain to find the percentage of error and further plotting it against option price

In [ ]:
fetch_option_data(stock_symbol,maturity_date_str)
errors = compare_prices(stock_symbol, maturity_date_str, fetch_current_risk_free_rate(), sigma=0)
plot_errors(errors)


In [ ]:

# Stress Testing
def stress_test(symbol, K, expiration_date):
    """
    Perform stress testing on Black-Scholes model.
    symbol: Stock ticker
    K: Strike price
    expiration_date: Option expiration date in 'YYYY-MM-DD' format
    """
    # Fetch stock data
    stock = yf.Ticker(symbol)
    current_price = get_stock_price(stock_symbol)
    expiration_date = np.datetime64(expiration_date)
    T = calculate_time_to_maturity(maturity_date_str)  # Time to maturity in years
    
    r = fetch_current_risk_free_rate() # Assume 5% risk-free rate
    call_sigma = call_option_IV(stock_symbol, K, maturity_date_str)
    put_sigma = put_option_IV(stock_symbol, K, maturity_date_str)
    # Calculate base prices
    call_price = black_scholes(current_price, K, T, r, call_sigma, option_type="call")
    put_price = black_scholes(current_price, K, T, r, put_sigma, option_type="put")

    print(f"Base Call Price: {call_price:.2f}, Base Put Price: {put_price:.2f}")

    # Stress testing variables
    stress_variables = {
        "Stock Price (S)": [current_price * 0.5, current_price * 2],
        "Volatility (σ)": [0.01, 1.0],
        "Time to Expiration (T)": [0.01, 5],
        "Risk-Free Rate (r)": [0, 0.2]
    }

    results = []

    # Stress testing each variable
    for variable, values in stress_variables.items():
        for value in values:
            if variable == "Stock Price (S)":
                S = value
                test_call_price = black_scholes(S, K, T, r, call_sigma, "call")
                test_put_price = black_scholes(S, K, T, r, put_sigma, "put")
            elif variable == "Volatility (σ)":
                test_call_price = black_scholes(current_price, K, T, r, value, "call")
                test_put_price = black_scholes(current_price, K, T, r, value, "put")
            elif variable == "Time to Expiration (T)":
                test_call_price = black_scholes(current_price, K, value, r, call_sigma, "call")
                test_put_price = black_scholes(current_price, K, value, r, put_sigma, "put")
            elif variable == "Risk-Free Rate (r)":
                test_call_price = black_scholes(current_price, K, T, value, call_sigma, "call")
                test_put_price = black_scholes(current_price, K, T, value, put_sigma, "put")
            
            results.append({
                "Variable": variable,
                "Value": value,
                "Call Price": test_call_price,
                "Put Price": test_put_price
            })
    print(results)
    # Plotting results
    for variable in stress_variables.keys():
        variable_results = [res for res in results if res["Variable"] == variable]
        values = [res["Value"] for res in variable_results]
        call_prices = [res["Call Price"] for res in variable_results]
        put_prices = [res["Put Price"] for res in variable_results]
        
        plt.figure(figsize=(10, 5))
        plt.plot(values, call_prices, label="Call Price", marker="o")
        plt.plot(values, put_prices, label="Put Price", marker="o")
        plt.title(f"Stress Testing: {variable}")
        plt.xlabel(variable)
        plt.ylabel("Option Price")
        plt.legend()
        plt.grid()
        plt.show()

Performing stress test on Black-Schole-Merton model to account for the chain in option price when we take one variable from the BSM equation to an extreme value. This is done to study variation in option prices in case of those low probablity, high impact market situations. 

In [ ]:
stress_test(stock_symbol, strike_price, maturity_date_str)

In [ ]:


def calculate_greeks(S, K, T, r, sigma):
    """
    Calculate the Greeks for an option.

    Parameters:
    - S: Current stock price
    - K: Strike price
    - T: Time to maturity (in years)
    - r: Risk-free rate (annualized)
    - sigma: Volatility (standard deviation of returns)

    Returns:
    - Greeks dictionary
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    delta_call = norm.cdf(d1)
    delta_put = norm.cdf(d1) - 1
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * np.sqrt(T) * norm.pdf(d1)
    theta_call = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))) - r * K * np.exp(-r * T) * norm.cdf(d2)
    theta_put = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))) + r * K * np.exp(-r * T) * norm.cdf(-d2)
    rho_call = K * T * np.exp(-r * T) * norm.cdf(d2)
    rho_put = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    return {
        "Delta (Call)": delta_call,
        "Delta (Put)": delta_put,
        "Gamma": gamma,
        "Vega": vega,
        "Theta (Call)": theta_call,
        "Theta (Put)": theta_put,
        "Rho (Call)": rho_call,
        "Rho (Put)": rho_put
    }

Calculating option greeks for given stock at a certain strike price for a specific expiration date

In [ ]:
greeks = calculate_greeks(get_stock_price(stock_symbol), strike_price, calculate_time_to_maturity(maturity_date_str), fetch_current_risk_free_rate(), call_option_IV(stock_symbol, strike_price,maturity_date_str))
print(greeks)

In [ ]:
# Step 1: Calculate Historical Volatility
def calculate_historical_volatility(ticker, period="6mo"):
    """
    Calculate annualized historical volatility for a given ticker.
    """
    try:
        data = yf.Ticker(ticker).history(period=period)
        data['Returns'] = np.log(data['Close'] / data['Close'].shift(1))
        hv = data['Returns'].std() * np.sqrt(252)  # Annualized volatility
        return hv
    except Exception as e:
        print(f"Error calculating historical volatility: {e}")
        return None

# Step 2: Fetch Implied Volatility
def get_implied_volatility(ticker, strike_price, expiration_date):
    """
    Fetch the implied volatility for a specific option.
    """
    try:
        iv = call_option_IV(ticker, strike_price, expiration_date)
        return iv
    except Exception as e:
        print(f"Error fetching implied volatility: {e}")
        return None

# Step 3: Combine HV and IV
def get_combined_volatility_range(ticker, strike_price, expiration_date, hv_period="6mo"):
    """
    Combine historical and implied volatility to create a range.
    """
    historical_volatility = calculate_historical_volatility(ticker, hv_period)
    implied_volatility = get_implied_volatility(ticker, strike_price, expiration_date)

    if historical_volatility is not None and implied_volatility is not None:
        low_end = min(historical_volatility, implied_volatility)
        high_end = max(historical_volatility, implied_volatility)
        volatility_range = np.linspace(low_end, high_end, 50
        )  # Create a range
        return volatility_range
    else:
        print("Unable to calculate a combined volatility range.")
        return None

Using historically volatility and implied volatility to have a combined volatility range

In [ ]:
volatility_range = get_combined_volatility_range(stock_symbol, strike_price,maturity_date_str)
if volatility_range is not None:
    print(f"Volatility Range: {volatility_range}")

In [ ]:
def plot_option_prices_and_greeks(S, K, T, r, sigma_range):
    """
    Plots the option price and Greeks as functions of volatility.

    Parameters:
    - S: Stock price
    - K: Strike price
    - T: Time to maturity
    - r: Risk-free rate
    - sigma_range: A list or array of volatility values
    """
    option_prices = []
    deltas = []
    gammas = []
    vegas = []
    thetas_call = []
    thetas_put = []
    print("Plotting greeks vs volatility graphs...")
    for sigma in sigma_range:
        price_call = black_scholes(S, K, T, r, sigma, option_type="call")
        greeks = calculate_greeks(get_stock_price(stock_symbol), strike_price, calculate_time_to_maturity(maturity_date_str), fetch_current_risk_free_rate(), sigma)

        option_prices.append(price_call)
        deltas.append(greeks["Delta (Call)"])
        gammas.append(greeks["Gamma"])
        vegas.append(greeks["Vega"])
        thetas_call.append(greeks["Theta (Call)"])
    print("option prices: ",option_prices)
    print("deltas: ", deltas)
    print("gammas: ",gammas)
    print("Vegas: ",vegas)
    print("thetas: ",thetas_call)
    print(thetas_put)
    # Create plots
    plt.figure(figsize=(12, 8))

    # Option Price vs. Volatility
    plt.subplot(2, 2, 1)
    plt.plot(sigma_range, option_prices, label="Call Option Price", color='blue')
    plt.title('Option Price vs. Volatility')
    plt.xlabel('Volatility')
    plt.ylabel('Option Price')

    # Delta vs. Volatility
    plt.subplot(2, 2, 2)
    plt.plot(sigma_range, deltas, label="Delta", color='green')
    plt.title('Delta vs. Volatility')
    plt.xlabel('Volatility')
    plt.ylabel('Delta')

    # Gamma vs. Volatility
    plt.subplot(2, 2, 3)
    plt.plot(sigma_range, gammas, label="Gamma", color='red')
    plt.title('Gamma vs. Volatility')
    plt.xlabel('Volatility')
    plt.ylabel('Gamma')

    # Vega vs. Volatility
    plt.subplot(2, 2, 4)
    plt.plot(sigma_range, vegas, label="Vega", color='purple')
    plt.title('Vega vs. Volatility')
    plt.xlabel('Volatility')
    plt.ylabel('Vega')

    # Display plot
    plt.tight_layout()
    plt.show()



def plot_option_prices_and_greeks_merged(S, K, T, r, sigma_range):
    """
    Plots the option price and Greeks as functions of volatility.
    Parameters:
    - S: Stock price
    - K: Strike price
    - T: Time to maturity
    - r: Risk-free rate
    - sigma_range: A list or array of volatility values
    """
    option_prices = []
    deltas = []
    gammas = []
    vegas = []
    thetas_call = []
    thetas_put = []
    print("Plotting greeks vs volatility graphs...")
    for sigma in sigma_range:
        price_call = black_scholes(S, K, T, r, sigma, option_type="call")
        greeks = calculate_greeks(get_stock_price(stock_symbol), strike_price, calculate_time_to_maturity(maturity_date_str), fetch_current_risk_free_rate(), sigma)

        option_prices.append(price_call)
        deltas.append(greeks["Delta (Call)"])
        gammas.append(greeks["Gamma"])
        vegas.append(greeks["Vega"])
        thetas_call.append(greeks["Theta (Call)"])
    print("option prices: ",option_prices)
    print("deltas: ", deltas)
    print("gammas: ",gammas)
    print("Vegas: ",vegas)
    print("thetas: ",thetas_call)
    print(thetas_put)
    # Create a single plot for all Greeks
    plt.figure(figsize=(10, 6))

    # Plot each Greek against volatility
    plt.plot(sigma_range, option_prices, label="Option Price", color='black', linestyle='-')
    plt.plot(sigma_range, deltas, label="Delta", color='green', linestyle='-')
    plt.plot(sigma_range, gammas, label="Gamma", color='red', linestyle='--')
    plt.plot(sigma_range, vegas, label="Vega", color='purple', linestyle='-.')
    plt.plot(sigma_range, thetas_call, label="Theta", color='blue', linestyle=':')

    # Add titles and labels
    plt.title("Option Greeks vs. Volatility")
    plt.xlabel("Volatility")
    plt.ylabel("Value")
    plt.legend()  # Show legend for all Greeks
    plt.grid(True)

    # Display plot
    plt.show()

Plotting each option greek as a function of combined volatility range to predict how option price may react to change in volatility in a probable range

In [ ]:
plot_option_prices_and_greeks(get_stock_price(stock_symbol), strike_price, calculate_time_to_maturity(maturity_date_str), fetch_current_risk_free_rate(), volatility_range)

In [ ]:
plot_option_prices_and_greeks_merged(get_stock_price(stock_symbol), strike_price, calculate_time_to_maturity(maturity_date_str), fetch_current_risk_free_rate(), volatility_range)

In [ ]:
def plt_greek(S, K, T, r, sigma, option_type='call'):
    """
    Calculates Greeks for options using the Black-Scholes model.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    delta = norm.cdf(d1) if option_type == 'call' else norm.cdf(d1) - 1
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)
    theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
             - r * K * np.exp(-r * T) * norm.cdf(d2) if option_type == 'call'
             else -S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
             + r * K * np.exp(-r * T) * norm.cdf(-d2))
    
    return delta, gamma, vega, theta

def plot_greeks_vs_prices_yf(stock_symbol, expiration_date,sigma_c,sigma_p,risk_free_rate):
    # Fetch the stock ticker
    stock = yf.Ticker(stock_symbol)
    
    try:
        # Get options for the specified expiration date
        options_data = stock.option_chain(expiration_date)
        calls = options_data.calls
        puts = options_data.puts
    except Exception as e:
        print(f"Error fetching option data: {e}")
        return
    
    # Ensure consistent strike prices between calls and puts
    common_strike_prices = calls.merge(puts, on="strike", suffixes=('_call', '_put'))['strike']
    
    # Get prices for only the common strikes
    call_prices = calls[calls['strike'].isin(common_strike_prices)]['lastPrice']
    put_prices = puts[puts['strike'].isin(common_strike_prices)]['lastPrice']

    # Ensure data alignment
    call_prices = call_prices.reset_index(drop=True)
    put_prices = put_prices.reset_index(drop=True)
    common_strike_prices = common_strike_prices.reset_index(drop=True)
    
    # Plotting Greeks vs. Prices
    plt.figure(figsize=(12, 8))
    
    # Plot Calls
    plt.plot(common_strike_prices, call_prices, label='Call Option Prices', marker='o')
    # Plot Puts
    plt.plot(common_strike_prices, put_prices, label='Put Option Prices', marker='o')
    
    # Get stock price and time to maturity
    S = stock.history(period='1d')['Close'].iloc[-1]
    T = (pd.to_datetime(expiration_date) - pd.Timestamp.today()).days / 365

    # Calculate Greeks
    call_greeks = [plt_greek(S, K, T, risk_free_rate, sigma_c, 'call') for K in common_strike_prices]
    put_greeks = [plt_greek(S, K, T, risk_free_rate, sigma_p, 'put') for K in common_strike_prices]

    # Extract individual Greeks
    call_delta, call_gamma, call_vega, call_theta = zip(*call_greeks)
    put_delta, put_gamma, put_vega, put_theta = zip(*put_greeks)

    # Plotting
    fig, axs = plt.subplots(4, 1, figsize=(12, 16), sharex=True)
    greeks = ['Delta', 'Gamma', 'Vega', 'Theta']
    call_data = [call_delta, call_gamma, call_vega, call_theta]
    put_data = [put_delta, put_gamma, put_vega, put_theta]

    for i, (greek, call_vals, put_vals) in enumerate(zip(greeks, call_data, put_data)):
        axs[i].plot(common_strike_prices, call_prices, label='Call Prices', color='blue')
        axs[i].plot(common_strike_prices, call_vals, label=f'Call {greek}', linestyle='--', color='green')
        axs[i].plot(common_strike_prices, put_prices, label='Put Prices', color='red')
        axs[i].plot(common_strike_prices, put_vals, label=f'Put {greek}', linestyle='--', color='orange')
        axs[i].set_ylabel(greek)
        axs[i].legend()
        axs[i].grid(True)

    axs[-1].set_xlabel('Strike Prices')
    fig.suptitle(f"Greeks vs Option Prices for {stock_symbol.upper()} (Expiration: {expiration_date})")
    plt.show()

Plotting each option greek against different strike prices to study how option price reacts to moneyness of the option

In [ ]:
plot_greeks_vs_prices_yf(stock_symbol, maturity_date_str,call_option_IV(stock_symbol, strike_price,maturity_date_str),put_option_IV(stock_symbol, strike_price,maturity_date_str),fetch_current_risk_free_rate())